In [1]:
##################################################################
########                                                  ########
########   Caderno Jupyter para fazer scrapp              ########
########   do diario oficial, e outros                    ########
########                                                  ########
########                                                  ########
##################################################################


In [2]:
# importando bibliotecas necessarias

import pandas as pd
import sqlalchemy
import pyodbc
import requests
import urllib.request
import time
import re
from bs4 import BeautifulSoup
from datetime import date
import json
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.image import MIMEImage



In [3]:
## metodo para enviar email

def enviarEmail(assunto, mensagem, destinatario):
    mailserver = smtplib.SMTP('smtp.df.cgu',25)
    mailserver.ehlo()
    mailserver.login('leandrors', 'padrao@2009')
    mailserver.sendmail('leandro.rangel@cgu.gov.br',destinatario,mensagem.as_string())
    mailserver.quit()



# parametros para o envio do email
subject = "BeTanIA - Leitura de DOU"
emailFrom = "leandro.rangel@cgu.gov.br"
emailTo = "leandro.rangel@cgu.gov.br"


htmlSessao1 = ""
htmlSessao2 = ""
htmlSessao3 = ""

In [4]:
## definicao das URLs

hoje = date.today()
hojeFormatado = hoje.strftime('%d-%m-%Y')

urlSessao1 = "http://www.in.gov.br/leiturajornal?data="+hojeFormatado+"&secao=do1"
#urlSessao1 = "http://www.in.gov.br/leiturajornal?data=15-07-2019&secao=do1"
urlInterno = "http://www.in.gov.br/web/dou/-/"

urlSessao2 = "http://www.in.gov.br/leiturajornal?data="+hojeFormatado+"&secao=do2"

urlSessao3 = "http://www.in.gov.br/leiturajornal?data="+hojeFormatado+"&secao=do3"

In [5]:
# Sessao 1
# com base na URL , fazemos a requsição e obtemos o JSON
response_sessao1 = requests.get(urlSessao1)
soup_sessao1 = BeautifulSoup(response_sessao1.text, "html.parser")
element_sessao1 = soup_sessao1.find("script", {"type":"application/json"})
json_sessao1 = json.loads(element_sessao1.text)   


In [6]:
## com base nos dados crus da sessao, vemos se tem conteúdo da CGU
## encontrando , adicionamos na lista de resposta
htmlSessao1+="<b>Seção 1.</b><br>"
urlsSessao1 = []
urlsSessao1INSS = []
for elemento in json_sessao1['jsonArray']:
    if "Controladoria" in elemento['hierarchyStr']:
        urlsSessao1.append(elemento['urlTitle'])
    if "Instituto Nacional do Seguro Social" in elemento['hierarchyStr']:
        urlsSessao1INSS.append(elemento['urlTitle'])
        
if len(urlsSessao1)>0:
    for title in urlsSessao1:
        htmlSessao1+="<b>CGU</b><br>"
        response_Interno1 = requests.get(urlInterno+title)
        soup_Interno1 = BeautifulSoup(response_Interno1.text, "html.parser")        
        identifica = soup_Interno1.find_all('p', {"class":"identifica"})    
        htmlSessao1+="<b>"+identifica[0].text+"</b><br>"
        paragrafo = soup_Interno1.find_all('p', {"class":"dou-paragraph"})    
        for i in paragrafo:
            htmlSessao1+=i.text+"<br>"
        htmlSessao1+="<br>"
else:
    htmlSessao1+="Não foi encontrada publicações da CGU na seção 1<br><br>"

if len(urlsSessao1INSS)>0:
    for title in urlsSessao1INSS:
        htmlSessao1+="<b>INSS</b><br>"
        response_Interno1 = requests.get(urlInterno+title)
        soup_Interno1 = BeautifulSoup(response_Interno1.text, "html.parser")        
        identifica = soup_Interno1.find_all('p', {"class":"identifica"})    
        htmlSessao1+="<b>"+identifica[0].text+"</b><br>"
        paragrafo = soup_Interno1.find_all('p', {"class":"dou-paragraph"})    
        for i in paragrafo:
            htmlSessao1+=i.text+"<br>"
        htmlSessao1+="<br>"
else:
    htmlSessao1+="Não foi encontrada publicações do INSS na seção 1<br><br>"
        

In [7]:
#Sessao 2 
# com base na URL , fazemos a requsição e obtemos o JSON
response_sessao2 = requests.get(urlSessao2)
soup_sessao2 = BeautifulSoup(response_sessao2.text, "html.parser")
element_sessao2 = soup_sessao2.find("script", {"type":"application/json"})
json_sessao2 = json.loads(element_sessao2.text)   


In [8]:
## com base nos dados crus da sessao, vemos se tem conteúdo da CGU
## encontrando , adicionamos na lista de resposta
htmlSessao2+="<b>Seção 2.</b><br>"
urlsSessao2 = []
urlsSessao2INSS = []

for elemento in json_sessao2['jsonArray']:
    if "Controladoria" in elemento['hierarchyStr']:
        urlsSessao2.append(elemento['urlTitle'])
    if "Instituto Nacional do Seguro Social" in elemento['hierarchyStr']:
        urlsSessao2INSS.append(elemento['urlTitle'])
          
if len(urlsSessao2)>0:
    for title in urlsSessao2:
        htmlSessao2+="<b>CGU</b><br>"
        response_Interno2 = requests.get(urlInterno+title)
        soup_Interno2 = BeautifulSoup(response_Interno2.text, "html.parser")        
        identifica2 = soup_Interno2.find_all('p', {"class":"identifica"})    
        htmlSessao2+="<b>"+identifica2[0].text+"</b><br>"
        paragrafo = soup_Interno2.find_all('p', {"class":"dou-paragraph"})    
        for i in paragrafo:
            htmlSessao2+=i.text+"<br>"
        htmlSessao2+="<br>"
else:
    htmlSessao2+="Não foi encontrada publicações da CGU na seção 2<br><br>"

    
if len(urlsSessao2INSS)>0:
    for title in urlsSessao2INSS:
        htmlSessao2+="<b>INSS</b><br>"
        response_Interno2 = requests.get(urlInterno+title)
        soup_Interno2 = BeautifulSoup(response_Interno2.text, "html.parser")        
        identifica2 = soup_Interno2.find_all('p', {"class":"identifica"})    
        htmlSessao2+="<b>"+identifica2[0].text+"</b><br>"
        paragrafo = soup_Interno2.find_all('p', {"class":"dou-paragraph"})    
        for i in paragrafo:
            htmlSessao2+=i.text+"<br>"
        htmlSessao2+="<br>"
else:
    htmlSessao2+="Não foi encontrada publicações do INSS na seção 2<br><br>"
    

In [9]:
#Sessao 3 
# com base na URL , fazemos a requsição e obtemos o JSON
response_sessao3 = requests.get(urlSessao3)
soup_sessao3 = BeautifulSoup(response_sessao3.text, "html.parser")
element_sessao3 = soup_sessao3.find("script", {"type":"application/json"})
json_sessao3 = json.loads(element_sessao3.text)   


In [10]:
## com base nos dados crus da sessao, vemos se tem conteúdo da CGU
## encontrando , adicionamos na lista de resposta
htmlSessao3+="<b>Seção 3.</b><br>"
urlsSessao3 = []
urlsSessao3INSS = []
for elemento in json_sessao3['jsonArray']:
    if "Controladoria" in elemento['hierarchyStr']:
        urlsSessao3.append(elemento['urlTitle'])
    if "Instituto Nacional do Seguro Social" in elemento['hierarchyStr']:
        urlsSessao3INSS.append(elemento['urlTitle'])
           
if len(urlsSessao3)>0:
    for title in urlsSessao3:
        htmlSessao3+="<b>CGU</b><br>"
        response_Interno3 = requests.get(urlInterno+title)
        soup_Interno3 = BeautifulSoup(response_Interno3.text, "html.parser")        
        identifica3 = soup_Interno3.find_all('p', {"class":"identifica"})    
        htmlSessao3+="<b>"+identifica3[0].text+"</b><br>"
        paragrafo = soup_Interno3.find_all('p', {"class":"dou-paragraph"})    
        for i in paragrafo:
            htmlSessao3+=i.text+"<br>"
        htmlSessao3+="<br>"
else:
    htmlSessao3+="Não foi encontrada publicações da CGU na seção 3<br><br>"

if len(urlsSessao3INSS)>0:
    for title in urlsSessao3INSS:
        htmlSessao3+="<b>INSS</b><br>"
        response_Interno3 = requests.get(urlInterno+title)
        soup_Interno3 = BeautifulSoup(response_Interno3.text, "html.parser")        
        identifica3 = soup_Interno3.find_all('p', {"class":"identifica"})    
        htmlSessao3+="<b>"+identifica3[0].text+"</b><br>"
        paragrafo = soup_Interno3.find_all('p', {"class":"dou-paragraph"})    
        for i in paragrafo:
            htmlSessao3+=i.text+"<br>"
        htmlSessao3+="<br>"
else:
    htmlSessao3+="Não foi encontrada publicações do INSS na seção 3<br><br>"
        

In [11]:
msg = MIMEMultipart("corpo")

msg['Subject'] = subject
msg['From'] = emailFrom
msg['To'] = emailTo

corpoHTML = MIMEText(htmlSessao1+htmlSessao2+htmlSessao3, 'html')

msg.attach(corpoHTML)

enviarEmail("DOU",msg,'leandro.rangel@cgu.gov.br')